In [6]:
!pip install fastapi nest-asyncio pyngrok uvicorn
!pip install pydantic
!pip install joblib
!pip install requests
!pip install pypi-json


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.3/99.3 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.1/127.1 kB 9.2 MB/s eta 0:00:00


In [7]:
!ngrok config add-authtoken 2l1Idz1luqOrx5yrph3QlJCbVTe_7M1q159Frd6LSiR3gUUDV

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [3]:
!pip install scikit_learn==1.3.2

In [8]:
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
import nest_asyncio
from pyngrok import ngrok
import uvicorn
from pydantic import BaseModel
import joblib as jb
import json
import pandas as pd

In [9]:
app = FastAPI()

In [10]:
origins = ["*"]

app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

In [11]:
class model_input(BaseModel):
  batting_team: str
  bowling_team: str
  city: str
  target: int
  balls_left: int
  wickets: int
  runs_scored: int

In [12]:
#loading the model
model = jb.load('/content/IPL_Win_Predictor.pkl')

In [13]:
@app.post('/model_prediction')
def model_prediction(input_parameters: model_input):
  input_data = input_parameters.json()
  input_dictionary = json.loads(input_data)
  batting_team = input_dictionary['batting_team']
  bowling_team = input_dictionary['bowling_team']
  city = input_dictionary['city']
  target = input_dictionary['target']
  balls_left = input_dictionary['balls_left']
  wickets = input_dictionary['wickets']
  runs_scored = input_dictionary['runs_scored']

  # making input DataFrame for Model
  new_df = pd.DataFrame({
        'batting_team' : [batting_team],
        'bowling_team' : [bowling_team],
        'city' : [city],
        'target' : [target],
        'balls_left': [balls_left],
        'wickets' : [wickets],
        'runs_left' : [(target- runs_scored)],
        'crr' : [(runs_scored)*6/(120 -balls_left)],
        'rrr' : [(target-runs_scored)*6/balls_left]
    })

  prediction = model.predict_proba(new_df)
  bowling_prob = prediction[0][0] * 100
  batting_prob = prediction[0][1] * 100
  return {
      batting_team : batting_prob,
      bowling_team : bowling_prob
  }


In [15]:
ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

Public URL: https://a809-34-32-180-212.ngrok-free.app


INFO:     Started server process [318]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     34.90.221.254:0 - "POST /model_prediction HTTP/1.1" 422 Unprocessable Entity
INFO:     34.90.221.254:0 - "POST /model_prediction HTTP/1.1" 422 Unprocessable Entity
INFO:     35.231.12.107:0 - "POST /model_prediction HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [318]
